In [34]:
import sys
sys.path.append("/planning_through_contact")

In [35]:
import numpy as np
import pandas as pd
import os
from examples.planar_hand.contact_sampler import PlanarHandContactSampler

from qsim.parser import QuasistaticParser

from irs_rrt.irs_rrt import IrsNode
from irs_rrt.irs_rrt_projection import IrsRrtProjection
from irs_rrt.rrt_params import IrsRrtProjectionParams, SmoothingMode
from examples.planar_hand.planar_hand_setup import *

from irs_mpc2.quasistatic_visualizer import QuasistaticVisualizer


In [36]:

# %% quasistatic system
q_parser = QuasistaticParser(q_model_path)

q_vis = QuasistaticVisualizer.make_visualizer(q_parser)
q_sim, q_sim_py = q_vis.q_sim, q_vis.q_sim_py
plant = q_sim.get_plant()

dim_x = q_sim.num_dofs()
dim_u = q_sim.num_actuated_dofs()
idx_a_l = plant.GetModelInstanceByName(robot_l_name)
idx_a_r = plant.GetModelInstanceByName(robot_r_name)
idx_u = plant.GetModelInstanceByName(object_name)
contact_sampler = PlanarHandContactSampler(
    q_sim=q_sim, q_sim_py=q_sim_py, pinch_prob=0.1
)

q_u0 = np.array([0.0, 0.35, 0])
q0_dict = contact_sampler.calc_enveloping_grasp(q_u0)
q0 = q_sim.get_q_vec_from_dict(q0_dict)


joint_limits = {
    idx_u: np.array([[-0.3, 0.3], [0.3, 0.5], [-0.01, np.pi + 0.01]])
}

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [37]:

# %% RRT testing
rrt_params = IrsRrtProjectionParams(q_model_path, joint_limits)
rrt_params.h = h
rrt_params.smoothing_mode = SmoothingMode.k1AnalyticPyramid
rrt_params.log_barrier_weight_for_bundling = 100
rrt_params.root_node = IrsNode(q0)
rrt_params.max_size = 100
rrt_params.goal = np.copy(q0)
rrt_params.goal[2] = np.pi
rrt_params.termination_tolerance = 0.01
rrt_params.goal_as_subgoal_prob = 0.3
rrt_params.regularization = 1e-4
rrt_params.rewire = False
rrt_params.grasp_prob = 0.
rrt_params.distance_threshold = np.inf
rrt_params.stepsize = 0.35
rrt_params.distance_metric = "local_u"

# params.distance_metric = 'global'  # If using global metric
rrt_params.global_metric = np.array([10.0, 10.0, 1.0, 0.1, 0.1, 0.1, 0.1])

print("start state:", q0)
print("goal state:", rrt_params.goal)

start state: [ 0.          0.35        0.         -0.88050497 -0.60592241  0.88050497
  0.60592241]
goal state: [ 0.          0.35        3.14159265 -0.88050497 -0.60592241  0.88050497
  0.60592241]


In [38]:
rrt_params.use_free_solvers = True
# contact_sampler.sim_params.use_free_solvers = True

In [39]:

prob_rrt = IrsRrtProjection(rrt_params, contact_sampler, q_sim, q_sim_py)
prob_rrt.iterate()

100%|█████████▉| 999/1000 [01:01<00:00, 16.21it/s]


In [40]:
(
    q_knots_trimmed,
    u_knots_trimmed,
) = prob_rrt.get_trimmed_q_and_u_knots_to_goal()
q_vis.publish_trajectory(q_knots_trimmed, h=rrt_params.h)

closest distance to subgoal 76.52210577809115


In [41]:
d_batch = prob_rrt.calc_distance_batch(rrt_params.goal)
print("minimum distance: ", d_batch.min())

# %%
node_id_closest = np.argmin(d_batch)

# %%
prob_rrt.save_tree(f"planar_hand_tree_{rrt_params.max_size}_{0}.pkl")


# %%
closest_state = prob_rrt.q_matrix[node_id_closest, :]
print("closest state", closest_state)
print("goal state", rrt_params.goal)


minimum distance:  76.52210577809115
closest state [-0.13355912  0.38456817  0.7661355   0.08969148 -1.90592598  1.26531252
  0.49808575]
goal state [ 0.          0.35        3.14159265 -0.88050497 -0.60592241  0.88050497
  0.60592241]


In [42]:
def compute_angular_error(angle, goal_angle):
    delta = 360 * (goal_angle - angle) / (2 * np.pi)
    delta_wraped = delta - (delta // 360) * 360
    if delta_wraped > 180:
        delta_wraped -= 360
    elif delta_wraped < -180:
        delta_wraped += 360
    angular_error = np.abs(delta_wraped)
    return angular_error

def compute_minimal_angular_error(prob_rrt):
    rrt_params = prob_rrt.rrt_params
    max_size = rrt_params.max_size
    goal_angle = rrt_params.goal[2]
    angular_errors = np.array([compute_angular_error(prob_rrt.q_matrix[i,2], goal_angle) for i in range(max_size)])
    return np.min(angular_errors)


def compute_closest_node_angular_error(prob_rrt):
    rrt_params = prob_rrt.rrt_params
    d_batch = prob_rrt.calc_distance_batch(rrt_params.goal)
    node_id_closest = np.argmin(d_batch)
    goal_angle = rrt_params.goal[2]
    angular_error = compute_angular_error(prob_rrt.q_matrix[node_id_closest,2], goal_angle)
    return angular_error


closest_node_angular_error = compute_closest_node_angular_error(prob_rrt)
minimal_angular_error = compute_minimal_angular_error(prob_rrt)

print("closest node angular error", closest_node_angular_error)
print("minimal angular error", minimal_angular_error)

closest node angular error 136.10366938594566
minimal angular error 121.9329133306245


In [43]:
# we benchmark the performanc eof the RRT search on 4 tasks: roate the ball by 45, 90, 135, and 180 degrees.
# we run 1400 iterations per RRT search (this corresponds to 1 min of compute per search on the bdai laptop)
# we run each task 50 times and report the average and std angular error for the ball.
# minimal_angular_error is the error of the state with minimal error in the tree
# closest_node_angular_error is the erro of the state considered closets to the goal

In [48]:
# goal_angles = np.array([45, 90, 135, 180]) / 360 * 2 * np.pi
# num_samples = 50
# num_iterations = 1400
goal_angles = np.array([45, 90]) / 360 * 2 * np.pi
num_samples = 4
num_iterations = 14


In [52]:
rrt_params.max_size = num_iterations

minimal_angular_errors = np.zeros((len(goal_angles), num_samples))
closest_node_angular_errors = np.zeros((len(goal_angles), num_samples))

for task_idx, goal_angle in enumerate(goal_angles):
    for sample_idx in range(num_samples):
        rrt_params.goal[2] = goal_angle
        prob_rrt = IrsRrtProjection(rrt_params, contact_sampler, q_sim, q_sim_py)
        prob_rrt.iterate()
        minimal_angular_errors[task_idx, sample_idx] = compute_minimal_angular_error(prob_rrt)
        closest_node_angular_errors[task_idx, sample_idx] = compute_closest_node_angular_error(prob_rrt)

print("minimal_angular_errors", minimal_angular_errors)
print("closest_node_angular_errors", closest_node_angular_errors)


 93%|█████████▎| 13/14 [00:00<00:00, 32.30it/s]

minimal_angular_errors [[42.16685087 42.48907939 43.06973948 42.17456388]
 [86.36314007 87.6954748  87.67725461 86.93100219]]
closest_node_angular_errors [[45. 45. 45. 45.]
 [90. 90. 90. 90.]]


In [50]:
# Convert matrices to DataFrame
df_minimal_angular_errors = pd.DataFrame(minimal_angular_errors)
df_closest_node_angular_errors = pd.DataFrame(closest_node_angular_errors)

# Insert goal_angles as the first column
df_minimal_angular_errors.insert(0, 'goal_angle', goal_angles * 360 / (2 * np.pi))
df_closest_node_angular_errors.insert(0, 'goal_angle', goal_angles * 360 / (2 * np.pi))

# Save DataFrames to CSV files

# Ensure the directory exists
benchmark_dir = "benchmark_data"
os.makedirs(benchmark_dir, exist_ok=True)

df_minimal_angular_errors.to_csv("benchmark_data/minimal_angular_errors.csv", index=False)
df_closest_node_angular_errors.to_csv("benchmark_data/closest_node_angular_errors.csv", index=False)

   goal_angle          0          1          2          3
0    0.785398  42.488130  42.346242  42.454353  42.701517
1    1.570796  87.709051  87.795023  87.553409  87.933081


ValueError: cannot insert goal_angle, already exists